In [1]:
from gbd_core.api import GBD
import pandas as pd

groups = []

with open("/home/raphael-zipperer/Uni/BA/git/Kissat_hyperparamoptimization/instances_families.txt") as targetfams:
    for line in targetfams:
        line = line.strip()
        line = line.split(" ")[1]
        fams = line.split(",")
        groups.append(fams)

def get_available_features():
    with GBD([ "/home/raphael-zipperer/Uni/BA/database/base.db" ]) as gbd:
        feat = gbd.get_features('base')
        feat.remove("base_features_runtime")
        feat.remove("ccs")
        feat.remove("bytes")
        return feat

def get_prediction_dataset(features, target):
    with GBD([ '/home/raphael-zipperer/Uni/BA/database/base.db', '/home/raphael-zipperer/Uni/BA/database/meta.db' ]) as gbd:
        df = gbd.query('(track=main_2023 or track=main_2024) and minisat1m!=yes', resolve=features+[target])
        df[features] = df[features].apply(pd.to_numeric)
        #for i, group in enumerate(groups):
        #    if df[target].isin(group).any():
        #        df[target] = df[target].apply(lambda x: i if x in group else x)
        return df

def get_dataset_by_hashes(hashes_list):
    with GBD(['/home/raphael-zipperer/Uni/BA/database/base.db', '/home/raphael-zipperer/Uni/BA/database/meta.db']) as gbd:
        features = get_available_features()
        all_hashes = [hash for sublist in hashes_list for hash in sublist]
        df = gbd.query('(track=main_2023 or track=main_2024) and minisat1m!=yes', resolve = features)
        df = df[df['hash'].isin(all_hashes)]
        df[features] = df[features].apply(pd.to_numeric)
        df['index'] = -1
        for i, sublist in enumerate(hashes_list):
            df.loc[df['hash'].isin(sublist), 'index'] = i
        return df
    
print(get_available_features())

['clauses', 'variables', 'cls1', 'cls2', 'cls3', 'cls4', 'cls5', 'cls6', 'cls7', 'cls8', 'cls9', 'cls10p', 'horn', 'invhorn', 'positive', 'negative', 'hornvars_mean', 'hornvars_variance', 'hornvars_min', 'hornvars_max', 'hornvars_entropy', 'invhornvars_mean', 'invhornvars_variance', 'invhornvars_min', 'invhornvars_max', 'invhornvars_entropy', 'balancecls_mean', 'balancecls_variance', 'balancecls_min', 'balancecls_max', 'balancecls_entropy', 'balancevars_mean', 'balancevars_variance', 'balancevars_min', 'balancevars_max', 'balancevars_entropy', 'vcg_vdegree_mean', 'vcg_vdegree_variance', 'vcg_vdegree_min', 'vcg_vdegree_max', 'vcg_vdegree_entropy', 'vcg_cdegree_mean', 'vcg_cdegree_variance', 'vcg_cdegree_min', 'vcg_cdegree_max', 'vcg_cdegree_entropy', 'vg_degree_mean', 'vg_degree_variance', 'vg_degree_min', 'vg_degree_max', 'vg_degree_entropy', 'cg_degree_mean', 'cg_degree_variance', 'cg_degree_min', 'cg_degree_max', 'cg_degree_entropy']


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA

feat = get_available_features()
data = get_prediction_dataset(feat, 'family')

X_train, X_test, y_train, y_test = train_test_split(data[feat], data['family'], test_size=0.2, random_state=42)
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

# Train a Decision Tree model
tree_model = DecisionTreeClassifier()
tree_model.fit(X_train, y_train)
tree_y_pred = tree_model.predict(X_test)
print("Decision Tree Accuracy:", accuracy_score(y_test, tree_y_pred))

# Train an SVM model
svm_model = SVC()
svm_model.fit(X_train, y_train)
svm_y_pred = svm_model.predict(X_test)
print("SVM Accuracy:", accuracy_score(y_test, svm_y_pred))

# Apply PCA on features
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train a Random Forest model on PCA-transformed data
model_pca = RandomForestClassifier()
model_pca.fit(X_train_pca, y_train)
y_pred_pca = model_pca.predict(X_test_pca)
print("PCA Random Forest Accuracy:", accuracy_score(y_test, y_pred_pca))

# Train a Decision Tree model on PCA-transformed data
tree_model_pca = DecisionTreeClassifier()
tree_model_pca.fit(X_train_pca, y_train)
tree_y_pred_pca = tree_model_pca.predict(X_test_pca)
print("PCA Decision Tree Accuracy:", accuracy_score(y_test, tree_y_pred_pca))

# Train an SVM model on PCA-transformed data
svm_model_pca = SVC()
svm_model_pca.fit(X_train_pca, y_train)
svm_y_pred_pca = svm_model_pca.predict(X_test_pca)
print("PCA SVM Accuracy:", accuracy_score(y_test, svm_y_pred_pca))




Accuracy: 0.9078947368421053
Decision Tree Accuracy: 0.8552631578947368
SVM Accuracy: 0.08552631578947369
PCA Random Forest Accuracy: 0.6118421052631579
PCA Decision Tree Accuracy: 0.5855263157894737
PCA SVM Accuracy: 0.08552631578947369


In [9]:
import pickle
import pandas as pd
from utils import *

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score

def flatten(t):
    if t.is_leaf():
        return [(t.config, t.hashes)]
    else:
        return flatten(t.left) + flatten(t.right)

with open('../results/toplevel/splits_8_dic.txt', 'r') as inp:
    configurations = []
    instances = []
    for line in inp:
        config_str, hashes_str ="a","b"
        if "Default" in line:
            config_str, hashes_str = line.split(": ")
            config = "Default"
        else:
            config_str, hashes_str = line.split("}: ")
            config = eval(config_str + "}")
        hashes = hashes_str.split(", ")
        configurations.append(config)
        instances.append(hashes)

    print(len(instances))

    feat = get_available_features()
    data = get_dataset_by_hashes(instances)
    
    X_train, X_test, y_train, y_test = train_test_split(data[feat], data['index'], test_size=0.2, random_state=42)

    # Count the number of instances of each index in the training set
    train_counts = y_train.value_counts()
    print("Training set counts:")
    print(train_counts)

    # Count the number of instances of each index in the entire dataset
    data_counts = y_test.value_counts()
    print("Dataset counts:")
    print(data_counts)
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    # Perform 5-fold cross-validation
    cv_scores = cross_val_score(RandomForestClassifier(), data[feat], data['index'], cv=5)
    print("5-Fold Cross-Validation Scores:", cv_scores)
    print("Mean CV Score:", cv_scores.mean())

    tree_model = DecisionTreeClassifier()
    tree_model.fit(X_train, y_train)
    tree_y_pred = tree_model.predict(X_test)
    print("Decision Tree Accuracy:", accuracy_score(y_test, tree_y_pred))

    # Save the Random Forest model to a file
    with open('random_forest_model.pkl', 'wb') as model_file:
        pickle.dump(model, model_file)

    # To access the saved model later, you can load it using the following code:
    # with open('random_forest_model.pkl', 'rb') as model_file:
    #     loaded_model = pickle.load(model_file)
    #     # Now you can use loaded_model to make predictions or evaluate

    




40
Training set counts:
index
3     68
0     29
11    24
7     22
12    22
1     19
24    16
22    14
23    14
18    14
6     13
5     11
30    11
20    10
28    10
9     10
17     9
31     9
33     8
35     8
29     8
21     8
26     8
25     8
10     8
27     8
2      7
13     7
34     7
15     7
32     7
39     7
8      6
38     6
36     6
4      6
37     5
16     4
14     4
19     4
Name: count, dtype: int64
Dataset counts:
index
3     12
7      9
0      7
11     7
12     6
23     5
18     5
22     5
1      4
4      4
6      4
36     4
15     4
14     4
37     3
29     3
19     3
16     3
24     3
8      3
5      2
9      2
35     2
2      2
31     2
39     2
32     2
28     1
13     1
27     1
38     1
25     1
20     1
17     1
Name: count, dtype: int64
Accuracy: 0.8403361344537815


KeyboardInterrupt: 

In [ ]:
with open('../results/toplevel/splits_8_dic.txt', 'r') as inp:
    configurations = []
    instances = []
    for line in inp:
        config_str, hashes_str ="a","b"
        if "Default" in line:
            config_str, hashes_str = line.split(": ")
            config = "Default"
        else:
            config_str, hashes_str = line.split("}: ")
            config = eval(config_str + "}")
        hashes = hashes_str.split(", ")
        configurations.append(config)
        instances.append(hashes)

    print(len(instances))

    feat = get_available_features()
    data = get_dataset_by_hashes(instances)
    
    model = RandomForestClassifier()
    model.fit(data[feat], data['index'])
    y_pred = model.predict(data[feat])
    print("Accuracy:", accuracy_score(data['index'], y_pred))
    with open('random_forest_full.pkl', 'wb') as model_file:
        pickle.dump(model, model_file)

40
Accuracy: 1.0


In [43]:
import pandas as pd
import pickle
from utils import *

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Load the CSV file
df = pd.read_csv('top15.csv')

# Find the configuration with the least time for each key
min_time_config = df.loc[df.groupby('key')['time'].idxmin()]

min_time_config = min_time_config[min_time_config['time'] <= 1800.0]

config_dict = {}
for _, row in min_time_config.iterrows():
    config_str = str(row['configuration'])
    if config_str not in config_dict:
        config_dict[config_str] = []
    config_dict[config_str].append(row['key'])


confs = []
insts = []
for key in config_dict.keys():
    insts.append(config_dict[key])
    confs.append(key)

feat = get_available_features()
data = get_dataset_by_hashes(insts)

X_train, X_test, y_train, y_test = train_test_split(data[feat], data['index'], test_size=0.2, random_state=42)
# Count the number of instances of each index in the training set
#train_counts = y_train.value_counts()

# Count the number of instances of each index in the entire dataset
#data_counts = y_test.value_counts()

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))



Accuracy: 0.46218487394957986


In [44]:
df = pd.read_csv('top15.csv')

# Find the configuration with the least time for each key
min_time_config = df.loc[df.groupby('key')['time'].idxmin()]

min_time_config = min_time_config[min_time_config['time'] <= 1800.0]

feat = get_available_features()
data = get_prediction_dataset(feat, "family")

family_hashes = {}
for family in data['family'].unique():
    family_hashes[family] = data[data['family'] == family]['hash'].tolist()
    family_hashes[family] = [h for h in family_hashes[family] if h in min_time_config['key'].values]
    if not family_hashes[family]:
        del family_hashes[family]


    # Create a dictionary to store the best configuration for each family
best_configurations = {}
# Iterate over each family in family_hashes
for family, hashes in family_hashes.items():

    # Filter the dataframe to include only the instances for the current family
    family_df = df[df['key'].isin(hashes)]
    
    # Group by configuration and sum the runtimes
    config_runtime_sum = family_df.groupby('configuration')['time'].sum()
    
    # Find the configuration with the minimum runtime sum
    best_config = config_runtime_sum.idxmin()
    
    # Store the best configuration for the current family
    best_configurations[family] = best_config

print(best_configurations)

# Reverse the dictionary: for each key, check which value is assigned to it.
# Assign the key to the value in a new dictionary
reversed_best_configurations = {}
for family, config in best_configurations.items():

    if config not in reversed_best_configurations:
        reversed_best_configurations[config] = []

    reversed_best_configurations[config].append(family)

print(reversed_best_configurations)

with open("top_15_conf.txt", "w") as file:
    for key in reversed_best_configurations.keys():
        file.write(key)
        file.write("\n")

hashes = []
for key in reversed_best_configurations.keys():
    families = reversed_best_configurations[key]

    nhash = []
    for fam in families:
        nhash.extend(family_hashes[fam].copy())
    hashes.append(nhash)

feat = get_available_features()
data = get_dataset_by_hashes(hashes)

X_train, X_test, y_train, y_test = train_test_split(data[feat], data['index'], test_size=0.2, random_state=42)
# Count the number of instances of each index in the training set
#train_counts = y_train.value_counts()

# Count the number of instances of each index in the entire dataset
#data_counts = y_test.value_counts()

model = RandomForestClassifier()
model.fit(data[feat], data['index'])

with open('random_forest_top15.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)




{'cellular-automata': "{'backbone': 2, 'bump': 1, 'chrono': 1, 'congruence': 0, 'eliminate': 0, 'extract': 1, 'factor': 1, 'fastel': 1, 'forward': 1, 'lucky': 0, 'phase': 0, 'phasesaving': 0, 'preprocess': 0, 'probe': 0, 'randec': 0, 'reluctant': 1, 'reorder': 0, 'rephase': 1, 'restart': 1, 'stable': 2, 'substitute': 0, 'sweep': 0, 'target': 2, 'transitive': 1, 'vivify': 0, 'warmup': 1}", 'hardware-verification': "{'backbone': 0, 'bump': 1, 'chrono': 1, 'congruence': 0, 'eliminate': 0, 'extract': 0, 'factor': 1, 'fastel': 1, 'forward': 1, 'lucky': 1, 'phase': 1, 'phasesaving': 1, 'preprocess': 1, 'probe': 1, 'randec': 0, 'reluctant': 0, 'reorder': 2, 'rephase': 0, 'restart': 1, 'stable': 0, 'substitute': 1, 'sweep': 1, 'target': 0, 'transitive': 0, 'vivify': 1, 'warmup': 0}", 'social-golfer': "{'backbone': 2, 'bump': 1, 'chrono': 1, 'congruence': 0, 'eliminate': 0, 'extract': 0, 'factor': 1, 'fastel': 1, 'forward': 0, 'lucky': 0, 'phase': 0, 'phasesaving': 1, 'preprocess': 0, 'probe': 

In [3]:
import pandas as pd
import pickle
from utils import *

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df = pd.read_csv('top40.csv')

# Find the configuration with the least time for each key
min_time_config = df.loc[df.groupby('key')['time'].idxmin()]

min_time_config = min_time_config[min_time_config['time'] <= 1800.0]

feat = get_available_features()
data = get_prediction_dataset(feat, "family")

family_hashes = {}
for family in data['family'].unique():
    family_hashes[family] = data[data['family'] == family]['hash'].tolist()
    family_hashes[family] = [h for h in family_hashes[family] if h in min_time_config['key'].values]
    if not family_hashes[family]:
        del family_hashes[family]


    # Create a dictionary to store the best configuration for each family
best_configurations = {}
# Iterate over each family in family_hashes
for family, hashes in family_hashes.items():

    # Filter the dataframe to include only the instances for the current family
    family_df = df[df['key'].isin(hashes)]
    
    # Group by configuration and sum the runtimes
    config_runtime_sum = family_df.groupby('configuration')['time'].sum()
    
    # Find the configuration with the minimum runtime sum
    best_config = config_runtime_sum.idxmin()
    
    # Store the best configuration for the current family
    best_configurations[family] = best_config

print(best_configurations)

# Reverse the dictionary: for each key, check which value is assigned to it.
# Assign the key to the value in a new dictionary
reversed_best_configurations = {}
for family, config in best_configurations.items():

    if config not in reversed_best_configurations:
        reversed_best_configurations[config] = []

    reversed_best_configurations[config].append(family)

print(reversed_best_configurations)

with open("top_40_conf.txt", "w") as file:
    for key in reversed_best_configurations.keys():
        file.write(key)
        file.write("\n")

hashes = []
for key in reversed_best_configurations.keys():
    families = reversed_best_configurations[key]

    nhash = []
    for fam in families:
        nhash.extend(family_hashes[fam].copy())
    hashes.append(nhash)

feat = get_available_features()
data = get_dataset_by_hashes(hashes)

X_train, X_test, y_train, y_test = train_test_split(data[feat], data['index'], test_size=0.2, random_state=42)
# Count the number of instances of each index in the training set
#train_counts = y_train.value_counts()

# Count the number of instances of each index in the entire dataset
#data_counts = y_test.value_counts()

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

with open('random_forest_top40.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

{'cellular-automata': "{'backbone': 2, 'bump': 1, 'chrono': 1, 'congruence': 0, 'eliminate': 0, 'extract': 1, 'factor': 1, 'fastel': 1, 'forward': 1, 'lucky': 0, 'phase': 0, 'phasesaving': 0, 'preprocess': 0, 'probe': 0, 'randec': 0, 'reluctant': 1, 'reorder': 0, 'rephase': 1, 'restart': 1, 'stable': 2, 'substitute': 0, 'sweep': 0, 'target': 2, 'transitive': 1, 'vivify': 0, 'warmup': 1}", 'hardware-verification': "{'backbone': 0, 'bump': 1, 'chrono': 1, 'congruence': 1, 'eliminate': 1, 'extract': 0, 'factor': 1, 'fastel': 0, 'forward': 1, 'lucky': 0, 'phase': 1, 'phasesaving': 1, 'preprocess': 0, 'probe': 1, 'randec': 1, 'reluctant': 1, 'reorder': 0, 'rephase': 0, 'restart': 1, 'stable': 0, 'substitute': 0, 'sweep': 1, 'target': 0, 'transitive': 0, 'vivify': 1, 'warmup': 1}", 'social-golfer': "{'backbone': 2, 'bump': 1, 'chrono': 1, 'congruence': 0, 'eliminate': 0, 'extract': 0, 'factor': 1, 'fastel': 1, 'forward': 0, 'lucky': 0, 'phase': 0, 'phasesaving': 1, 'preprocess': 0, 'probe': 